In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# drive.flush_and_unmount()
# print('All changes made in this colab session should now be visible in Drive.')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow # colab does not support cv2.imshow()

# Download & compile the YOLO Darknet network into Google Colab



In [ ]:
%cd "/content"

/content


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
%pwd

'/content'

In [ ]:
%cd ./darknet

/content/darknet


In [ ]:
!chmod +rwx ./*

Open the **makefile** file and change these lines

GPU=1

CUDNN=1

OPENCV=1

`ARCH= -gencode arch=compute_20,code=[sm_20,sm_21]` \
-gencode arch=compute_30, code=...

For K80 GPU

`ARCH= -gencode arch=compute_37, code=sm_37`

In [ ]:
!make

# Build the Network Config file

1.  Copy the yolov4.cfg file an create this new copy: document_yolov4.cfg
2.  Open and modify the file with these entries :

    Line 3 : subdivisions=12
    
    Line 7: width=416

    Line 8: height=416

    Line 19: max_batches = 2000

    Line 21: steps=1600,1800

    Line 961,1049,1137: filters=18 # (classes+5)*3
    
    Line 968,1056,1144: classes=1


# Test the YOLO network


In [ ]:
!./darknet detector test cfg/coco.data "/content/darknet/doc_data/yolov4.cfg"  "/content/darknet/doc_data/yolov4.weights"  "/content/darknet/doc_data/yoloimg.jpg"


 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
Couldn't open file: /content/darknet/doc_data/yolov4.cfg


# Prepare the custom model

1. Get images from any sources ...
2. Label images by using for example LabelImg (https://github.com/tzutalin/labelImg)

After labeling all your images (here i'll label only 300 images) you should have 3 kind of files:

- the images itself (jpg, png, etc.)
- a set of text file (one per image) associated with the coordinates of the boxes in the image (thes files are generated by LabelImg)
- a classes file which list the classes

Here the images are located in /content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data/doc_images

# Configuration files
## Split the dataset (Train/Test)

1. Create one file with the list of 80% of the images (train) : 240 images in our case (document_training.txt)
2. Create one file with the list of 20% of the images (test) : 60 images in our case (document_test.txt)
3. Create a folder named doc_data into the root of darknet (/content/drive/MyDrive/Colab Notebooks/YOLO/darknet/doc_data)
4. Put the two files in this directory

In [ ]:
from os import listdir
from os.path import isfile, join
mydirectory = "/content/darknet/doc_data/doc_images"
allfiles = [f for f in listdir(mydirectory) if isfile(join(mydirectory, f))]

In [ ]:
len(allfiles)

265

In [ ]:
rootdir = "doc_data/doc_images/"
counter = 1
images_files = []

# put all the images files into a list
for file in allfiles:
  if (file[len(file)-3:len(file)] != "txt"):
    images_files.append(rootdir +  file)

# Set the size of the test (20%) & train data (80%)
training_limit = int(len(images_files) * 80 / 100)
test_limit = len(images_files) - training_limit

# Build the two lists
images_training_files = []
images_test_files = []
for file in images_files:
  if (counter <= training_limit):
    images_training_files.append(file)
  else:
    images_test_files.append( file)
  counter = counter + 1

# Put the two list into two files
with open('/content/darknet/doc_data/document_test.txt', 'w') as f:
    for item in images_test_files:
        f.write("%s\n" % item)

with open('/content/darknet/doc_data/document_training.txt', 'w') as f:
    for item in images_training_files:
        f.write("%s\n" % item)

# List of labels/classes
1. Then create a file named document.names and put the label (only one here) in there.

# Configuration bundle file
Create a file named document.data which will regroup all the configuration. Put that content inside :


classes = 1

valid = doc_data/document_test.txt

train = doc_data/document_training.txt

names = doc_data/document.names

backup = backup

# Train the model

In [ ]:
%cd "/content/darknet"
!chmod +rwx ./*

/content/darknet


In [ ]:
# !./darknet detector train doc_data/document.data doc_data/yolov4_custom.cfg doc_data/yolov4.conv.137 -dont_show -map

In [ ]:
!./darknet detector train doc_data/document.data doc_data/yolov4_custom.cfg backup/yolov4_custom_last.weights -dont_show -map


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Loaded: 0.000077 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.752621), count: 184, class_loss = 13.097992, iou_loss = 351.152008, total_loss = 364.250000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.717783), count: 4, class_loss = 0.709209, iou_loss = 2.229413, total_loss = 2.938622 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 652726, rewritten_bbox = 0.015627 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.711312), count: 119, class_loss = 11.709883, iou_loss = 198.141525, total_loss = 209.851410 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.692884), count: 9, class_loss = 1.437181, iou_loss = 3.139579, total_loss